In [ ]:
import matlab.engine
import cv2 as cv
eng = matlab.engine.start_matlab()
print('Finish starting matlab engine ...')


In [ ]:
im1 = eng.im2uint8(eng.imread('opt1.jpg'))
im2 = eng.im2uint8(eng.imread('opt2.jpg'))

print("Finish reading images ...")

In [ ]:
key1 = eng.FeatureDetection(im1, 4.0, 6.0, 5000.0)
print(key1.size)

key2 = eng.FeatureDetection(im2, 4.0, 6.0, 5000.0)
print("Finish feature detection ...")


In [ ]:
kpts1 = eng.kptsOrientation(key1, m1, 1, 96)
kpts2 = eng.kptsOrientation(key2, m2, 1, 96)
print('Finish main orientation calculation ...')

In [ ]:
des1 = eng.FeatureDescribe(im1, eo1, kpts1, 96, 6, 6)
des2 = eng.FeatureDescribe(im2, eo2, kpts2, 96, 6, 6)
print('Finish feature description ...')

In [ ]:
indexPairs,matchmetric = eng.matchFeatures(des1',des2','MaxRatio',1,'MatchThreshold', 100)
kpts1 = kpts1.T
kpts2 = kpts2.T
matchedPoints1 = kpts1[indexPairs[:, 1], 1:2]
matchedPoints2 = kpts2[indexPairs[:, 2], 1:2]
[matchedPoints2,IA]=eng.unique(matchedPoints2,'rows')
matchedPoints1=matchedPoints1[IA,:]
print('Finish feature matching ...')

In [ ]:
H=eng.FSC(matchedPoints1,matchedPoints2,'similarity',3);
Y_=H*[matchedPoints1.T;ones(1,size(matchedPoints1,1))];
Y_[1,:]=Y_[1,:]./Y_[3,:]
Y_[2,:]=Y_[2,:]./Y_[3,:]
E=eng.sqrt(eng.sum(Y_[1:2,:]-matchedPoints2.T).^2)
inliersIndex=E<3
cleanedPoints1 = matchedPoints1[inliersIndex, :]
cleanedPoints2 = matchedPoints2[inliersIndex, :]
print('Finish outlier removal ...')